In [1]:
# !pip install konlpy
# !pip install gensim
# !pip install fasttext
# import fasttext

In [33]:
import keras

import tensorflow
print(keras.__version__)
print(tensorflow.__version__)

2.3.1
1.13.2


In [98]:
from konlpy.tag import Okt
from gensim.models import Word2Vec
# from gensim.models import FastText
import pandas as pd
import re
import numpy as np
from sklearn.preprocessing import StandardScaler

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)


In [99]:
import math
def delete_emoji(sentence):
    try:
        hangul = re.compile('[^ ㄱ-ㅣ가-힣A-Za-z0-9]+') # 한글과 띄어쓰기를 제외한 모든 글자
        result = hangul.sub('', sentence) # 한글과 띄어쓰기를 제외한 모든 부분을 제거
#         result.replace('\n',' ')
        return result
    except:
        print(sentence)
        return ' '



In [9]:

df = pd.read_csv('data/except_moviereviews.csv')
df2 = pd.read_csv('data/except_moviereviews_20191107.csv')


In [32]:
df.shape

(1795, 3)

In [14]:
df3 = pd.concat([df,df2])

In [18]:
df4 = df3.drop_duplicates()

In [20]:
df4['subject'].value_counts()

1    930
2    806
3    781
Name: subject, dtype: int64

In [21]:
df = df4

In [22]:

%%time
import urllib
from bs4 import BeautifulSoup
title=[]
for idx, url in enumerate(list(df['url'])):
    if idx%10==0:
        print(idx)
    target_html = 'https://www.youtube.com{}'.format(url)
    html = urllib.request.urlopen(target_html).read()
    soup = BeautifulSoup(html, 'html.parser')
    title.append(soup.find('title').text[:-10])


0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [55]:
df['title']=title

In [24]:
df.head()

,url,subject,title
0,/watch?v=vdCSoJ8w4hk,1,칼리스타 창으로 아군 서포터 죽여버림ㅋㅋㅋ
1,/watch?v=0z0gj6BUmD8,1,맨날 화나있던 그가 서폿칭찬을 한 이유【천상계 랭겜】
2,/watch?v=xTTZGtgL5OA,1,아군 정글러에게 매운맛을 보여줬습니다.
3,/watch?v=6TSiO7Y0p2A,1,"지금 바루스를 해야하는 이유. ""숨은 꿀챔"""
4,/watch?v=C30FhUGTVnY,1,극한의 이득을위해 일부러(?) 죽는 이즈리얼


In [62]:
df.to_csv('title_label_except_moviereviews_20191107.csv',encoding='utf-8',index=False)

In [154]:
df = pd.read_csv('title_label_except_moviereviews_20191107_2.csv')
df.dropna(inplace=True)
# df2 = pd.read_csv('data/title_label2.csv')

In [155]:
df['subject'].value_counts()

1    915
2    788
3    741
Name: subject, dtype: int64

In [156]:
print(df.shape)
df.drop_duplicates(inplace=True)
print(df.shape)

(2444, 3)
(2444, 3)


In [157]:
# remove=[]
# for idx, i in enumerate(list(df['title'])[:100]):
#     subject = list(df['subject'])[idx]
#     if subject==1:
#         s='게임'
#     elif subject==2:
#         s='노래'
#     elif subject==3:
#         s='IT'
#     elif subject==4:
#         s='영화'
#     print("[{}] {}".format(s,i))
#     r = input()
#     if r=='1':
#         remove.append(1)
#     else:
#         remove.append(0)

In [158]:
# df_test = df.loc[ ((df['subject'] == 1) | (df['subject']==2))]

In [159]:
%%time
tagger = Okt()
sentences = list(df['title'])
sentences = [delete_emoji(sentence) for sentence in sentences]
sentences_Okt = []
for idx, sentence in enumerate(sentences):
    sen_okt = tagger.morphs(sentence)
    sentences_Okt.append(sen_okt)

Wall time: 1.96 s


In [14]:
# word2vec_model = Word2Vec(sentences=sentences_Okt, size=300, window=4, min_count=1, sg=1)

In [15]:
%%time
from gensim import models
# models.fast
# cap_path = datapath("C:/Users/JeongMyeong/Desktop/word-embeddings/fasttext/fasttext.bin")
fb_model = models.fasttext.load_facebook_model('C:/Users/JeongMyeong/Desktop/word-embeddings/fasttext/fasttext.bin')

Wall time: 14.6 s


In [107]:
word2vec_model = Word2Vec.load('C:/Users/JeongMyeong/Desktop/word-embeddings/word2vec/word2vec')

In [182]:
from sklearn.externals.joblib import dump, load
def vectorize(model, sentences):
  
    result = []
    for idx, sentence in enumerate(sentences):
        tmp = []
        for word in sentence:
            try:
                tmp.append(model.wv.word_vec(word))
            except:
                pass
        if tmp==[]:
            print(sentence)
            result.append(np.zeros((model.wv.vector_size, ),dtype='float32'))
        else:
            result.append(sum(tmp)/len(tmp)) 
    return np.array(result)


def scaler_transform(train, test): # , valid, test):
    scaler = StandardScaler()
#     scaler = load('std_scaler.bin')
    train_scale = scaler.fit_transform(train)
#     valid_scale = scaler.transform(valid)
    test_scale = scaler.transform(test)
    dump(scaler, 'std_scaler_20191107.bin', compress=True)
    return train_scale, test_scale # valid_scale, test_scale

def score(prediction, label):
    prediction = [1 if p>0.5 else 0 for p in prediction]
    return accuracy_score(prediction, label)

In [183]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(sentences_Okt, df['subject'], test_size=0.3, random_state=42)
X_train_vec = vectorize(fb_model, X_train)
# X_valid_vec = vectorize(word2vec_model, X_valid_Okt)
X_test_vec = vectorize(fb_model, X_test)

In [184]:
dim = fb_model.wv.vector_size
vocab = len(fb_model.wv.vocab)

In [185]:
import keras
X_train_scale, X_test_scale = scaler_transform(X_train_vec, X_test_vec)
y_train_category = [i-1 for i in y_train]
y_test_category = [i-1 for i in y_test]
y_train_category = keras.utils.np_utils.to_categorical(y_train_category, num_classes=3)
y_test_category = keras.utils.np_utils.to_categorical(y_test_category, num_classes=3)

X_train_scale, X_valid_scale, y_train_category, y_valid_category = train_test_split(X_train_scale, y_train_category, test_size=0.15, random_state=42)

In [186]:
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from keras.models import Model 
from keras.layers import Flatten, Dropout,Conv2D, concatenate,Conv1D, MaxPooling1D,Dense, Embedding, LSTM, Input
from sklearn.preprocessing import StandardScaler
from keras.callbacks import EarlyStopping
# import tensorflow as tf
# run_opts = tf.RunOptions(report_tensor_allocations_upon_oom = True)


input_sentence = Input(shape=(dim,))
emb = Embedding(vocab+1, 32, input_length=dim)(input_sentence)
drop = Dropout(0.3)(emb)
conv = Conv1D(3,3, padding='valid', activation='relu')(drop)
maxpool = MaxPooling1D(pool_size=1)(conv)
flat = Flatten()(maxpool)
hidden = Dense(16, activation='relu')(flat)
drop = Dropout(0.5)(hidden)
output = Dense(3, activation='softmax')(drop)
cnn_model = Model(input_sentence, output)
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])#, options = run_opts)
early_stopping  = EarlyStopping()


In [187]:
cnn_model.fit(X_train_scale,y_train_category, 
            batch_size=32, 
            epochs=30, 
            callbacks=[early_stopping],
            validation_data=(X_valid_scale, y_valid_category)
           )

Train on 1453 samples, validate on 257 samples
Epoch 1/30
1453/1453 [==============================] - 1s 479us/step - loss: 1.0978 - accuracy: 0.3565 - val_loss: 1.0948 - val_accuracy: 0.4047
Epoch 2/30
1453/1453 [==============================] - 0s 131us/step - loss: 1.0911 - accuracy: 0.3944 - val_loss: 1.0774 - val_accuracy: 0.5253
Epoch 3/30
1453/1453 [==============================] - 0s 131us/step - loss: 1.0630 - accuracy: 0.4639 - val_loss: 1.0090 - val_accuracy: 0.5992
Epoch 4/30
1453/1453 [==============================] - 0s 136us/step - loss: 0.9693 - accuracy: 0.5733 - val_loss: 0.8139 - val_accuracy: 0.7432
Epoch 5/30
1453/1453 [==============================] - 0s 133us/step - loss: 0.7822 - accuracy: 0.6889 - val_loss: 0.5753 - val_accuracy: 0.8366
Epoch 6/30
1453/1453 [==============================] - 0s 131us/step - loss: 0.6341 - accuracy: 0.7715 - val_loss: 0.4315 - val_accuracy: 0.8833
Epoch 7/30
1453/1453 [==============================] - 0s 133us/step - loss:

In [197]:
cnn_model.save('classification_three_20191107.h5')

In [198]:
predict = cnn_model.predict(X_test_scale)

In [209]:
predict[17]*100

array([69.11544 , 16.151625, 14.732938], dtype=float32)

In [210]:
prediction=[]
for i in range(len(X_test_scale)):
    tt = list(predict[i])
    prediction.append(tt.index(max(tt)))
    

In [211]:
prediction = keras.utils.np_utils.to_categorical(prediction, num_classes=3)

In [191]:
accuracy_score(prediction, y_test_category)

0.9100817438692098

In [192]:
from sklearn.metrics import multilabel_confusion_matrix

In [193]:
from sklearn.metrics import confusion_matrix

In [194]:
pred = np.argmax(prediction, axis=1).reshape(-1,1)
y_test_category_argmax = np.argmax(y_test_category, axis=1).reshape(-1,1)

In [195]:
# 게임, 음악, IT
confusion_matrix(pred, y_test_category_argmax)

array([[265,  13,  10],
       [  9, 208,   6],
       [ 23,   5, 195]], dtype=int64)

In [174]:
# 게임, 음악, IT
confusion_matrix(pred, y_test_category_argmax)

array([[270,  19,  23],
       [  8, 201,  10],
       [ 19,   6, 178]], dtype=int64)

In [196]:
cnt=0
for idx, (predict, label) in enumerate(zip(pred, y_test_category_argmax)):
    if predict==2 and label==0:
        print(X_test[idx], predict, label)
        cnt+=1
print(cnt, len(y_test_category))
print(cnt/len(y_test_category))

# 게임, 음악, IT

['대', '도서관', 'PS', 'VR', '으악', '상어', '가', '나타났다', 'VR', '월드', '오션', '디센트', 'PS', 'VR', 'Worlds', 'Ocean', 'Descent'] [2] [0]
['룰러', '의', '역', '관광', '무빙', '원딜', '룰러', '11월', '롤', '매드무비', 'GEN', 'G', 'Ruler', 'Montage'] [2] [0]
['대통령', '경호', '하기', '게임', '대', '도서관', '코믹', '실황', '1', '화', 'Mr', 'President'] [2] [0]
['진짜', '말', '도', '안되는', '옵션', '이', '떠', '버렸습니다', '팡', '이요', '메이플스토리'] [2] [0]
['코브', '4', 'K', 'ㅣ', '세계', '최강', '의', '그래픽', '을', '가진', '게임', 'TOP', '10', '파트', '1'] [2] [0]
['신규', '빠칭코', '1등', '당첨', '됐다', 'ㄷㄷ', '마일리지', '상품', '까지', '몸빵', '현질', '피파', '4'] [2] [0]
['도적', '전', '직업', '5초', '쿨', '감모', '자', '성능', '실험', '해봤습니다', '메이플스토리', '후니'] [2] [0]
['정글', '동선', '설계', '법', '강의', '정글', '심화', '분석', '동선', '낭비', '0', '챌', '린', '저', '마스터', '동선', '설계', '패턴', '분석', '롤', '정글', '운영', '법라', '인', '비중', '라인', '이해', '라인', '흐름', '구간', '별', '예측'] [2] [0]
['급발진', '역대', '급', '김여사', '주차', '된', '포르쉐', '차량', '박살', 'CCTV', '영상', '확보'] [2] [0]
['역대', '급', '탑신', '병자', '를', '만났습니다'] [2] [0]
['Chovy', '조이', 

In [36]:
from sklearn.metrics import multilabel_confusion_matrix

In [86]:
from tensorflow.python.client import device_lib

print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8102665146796905542
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6600259666
locality {
  bus_id: 1
  links {
  }
}
incarnation: 13732534684489279877
physical_device_desc: "device: 0, name: GeForce RTX 2070 SUPER, pci bus id: 0000:07:00.0, compute capability: 7.5"
]
